In [3]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client

In [4]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.17.5
0.23.0


In [5]:
%%timeit
import os
env = %env

83.9 µs ± 6.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [24]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS/"
features_dir = "../createFeatures/"
check_dir = "../CheckAvailableData/"
loadLas_dir = "../loadLAS/"


In [25]:
####
file_with_wells = 'dict_of__wells_df_No_features_class3_20180919a.p'

## Import all the data

In [32]:
##df_1.to_hdf('df_all_wells_noKNNorFeatures_20180920_a.h5', key='df', mode='w')
df_all_Col_preSplit = pd.read_hdf(load_dir+'df_all_wells_noKNNorFeatures_20180920_a.h5', 'df')

In [33]:
####
#wells_available = pd.read_pickle(loadLas_dir+file_with_wells)

In [34]:
#len(wells_available)

In [35]:
#well_file_names = list(wells_available.keys())

In [36]:
#type(well_file_names)

In [37]:
#well_file_names[0:5]

## Split data into train & test on a well by well basis, not row basis

In [38]:
UWIs = list(df_all_Col_preSplit['UWI'].unique())

In [39]:
numberOfTrainingWells = math.floor(len(UWIs)*0.8)
numberOfTrainingWells

1457

Randomly select that number of UWIs for training and the ones left for test

In [41]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [42]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [43]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1457
test 365


In [44]:
df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [45]:
df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [46]:
df_all_Col_preSplit_wTrainTest.tail()

,CALI,COND,DELT,DEPT,DPHI,DT,GR,ILD,ILM,NPHI,PHID,RHOB,SFL,SFLU,SN,SP,UWI,trainOrTest
1016,NaN,NaN,NaN,589.00,0.141,NaN,21.793,12.235,NaN,0.293,NaN,NaN,NaN,NaN,NaN,NaN,00-02-05-074-04W5-0.LAS,train
1017,NaN,NaN,NaN,589.25,0.134,NaN,22.354,12.235,NaN,0.276,NaN,NaN,NaN,NaN,NaN,NaN,00-02-05-074-04W5-0.LAS,train
1018,NaN,NaN,NaN,589.50,0.131,NaN,22.343,12.236,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,00-02-05-074-04W5-0.LAS,train
1019,NaN,NaN,NaN,589.75,0.126,NaN,22.327,12.462,NaN,0.252,NaN,NaN,NaN,NaN,NaN,NaN,00-02-05-074-04W5-0.LAS,train
1020,NaN,NaN,NaN,590.00,0.123,NaN,22.311,12.692,NaN,0.245,NaN,NaN,NaN,NaN,NaN,NaN,00-02-05-074-04W5-0.LAS,train


## Save to file

In [47]:
df_all_Col_preSplit_wTrainTest.to_hdf('df_all_wells_noKNNorFeatures_wTrainSplit_20180920_a.h5', key='df', mode='w')